In [136]:
pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


    ERROR: Complete output from command python setup.py egg_info:
    ERROR: Traceback (most recent call last):
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-4yud2msm\ruamel-yaml-conda\ruamel_yaml\__init__.py", line 21, in <module>
        from .main import *                               # NOQA
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-4yud2msm\ruamel-yaml-conda\ruamel_yaml\main.py", line 12, in <module>
        import ruamel.yaml
    ModuleNotFoundError: No module named 'ruamel'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-4yud2msm\ruamel-yaml-conda\setup.py", line 14, in <module>
        import ruamel_yaml  # NOQA
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-4yud2msm\ruamel-yaml-conda\ruamel_yaml\__init__.py", line 23, in <module>
        from ruamel_yaml.main import *

In [137]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk
import re

from top2vec import Top2Vec


# nltk.download('punkt')
# from nltk.tokenize import word_tokenize

# Topic Modeling Using Top2Vec

Load NYT Data for 2020 and 2021

In [3]:
nyt2020 = pd.read_csv("data/NYTData2020.csv")
nyt2021 = pd.read_csv("data/NYTData2021.csv")

nyt = nyt2021.append(nyt2020)

Take NYT review summaries and train a Top2Vec Model

https://pypi.org/project/top2vec/

In [4]:
documents = nyt['summary_short'].tolist()

model = Top2Vec(documents, speed="deep-learn", workers=8,
                embedding_model = "universal-sentence-encoder")

2022-01-10 19:38:04,730 - top2vec - INFO - Pre-processing documents for training
2022-01-10 19:38:05,084 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:absl:Using C:\Users\kenda\AppData\Local\Temp\tfhub_modules to cache modules.
2022-01-10 19:38:22,116 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2022-01-10 19:38:23,597 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2022-01-10 19:38:52,473 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2022-01-10 19:38:52,640 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


Print Number of Topics Defined by Model

In [5]:
model.get_num_topics()

3

How many documents are most similar to each topic?

In [6]:
topic_sizes, topic_nums = model.get_topic_sizes()

In [7]:
topic_sizes

array([930, 411,  55], dtype=int64)

Define Top Words for Each Topic

In [8]:
topic_words, word_scores, topic_nums = model.get_topics(model.get_num_topics())

In [9]:
topic_words

array([['comedy', 'drama', 'thriller', 'director', 'movie', 'film',
        'documentary', 'this', 'netflix', 'into', 'that', 'plays', 'it',
        'story', 'the', 'her', 'and', 'as', 'of', 'from', 'its', 'stars',
        'but', 'family', 'is', 'with', 'about', 'for', 'an', 'to', 'in',
        'feature', 'who', 'his', 'at', 'two', 'new', 'by', 'on',
        'follows', 'young', 'are', 'woman', 'life', 'up', 'their'],
       ['documentary', 'thriller', 'film', 'drama', 'that', 'this',
        'movie', 'comedy', 'about', 'it', 'director', 'life', 'into',
        'from', 'story', 'the', 'her', 'of', 'and', 'as', 'but',
        'netflix', 'woman', 'who', 'feature', 'with', 'for', 'its', 'an',
        'is', 'to', 'on', 'two', 'at', 'by', 'family', 'in', 'up', 'his',
        'are', 'plays', 'new', 'stars', 'their', 'young', 'follows'],
       ['documentary', 'netflix', 'movie', 'film', 'thriller', 'comedy',
        'drama', 'her', 'director', 'this', 'that', 'it', 'as', 'from',
        'the'

-----

# Topic Modeling with OMDB Plots

Load in OMDB Data

In [10]:
omdb2020 = pd.read_csv("data/OMDBData2020.csv")
omdb2020.Plot = omdb2020.Plot.fillna(value="")

omdb2021 = pd.read_csv("data/OMDBData2021.csv")
omdb2021.Plot = omdb2021.Plot.fillna(value="")

omdb = omdb2020.append(omdb2021)

Train TFIDF Vectorizer

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

tfidf_vectorizer_NMF = TfidfVectorizer(max_features = 20000, # only top 5k by freq
                                       lowercase = True, # drop capitalization
                                       ngram_range = (1,2), # include up to 2-grams, we can make this only 1 if needed
                                       min_df=1,  # note: absolute count of doc
                                       token_pattern = r'\b[a-z]{3,12}\b',   # remove short, non-word-like terms
                                       stop_words='english') # default English stopwords

tfidf_documents_NMF = tfidf_vectorizer_NMF.fit_transform(omdb["Plot"])
feature_names_NMF = tfidf_vectorizer_NMF.get_feature_names_out()

Train NMF for n = 2-10 topics.

In [12]:
from sklearn.decomposition import NMF
for num_topics in range(2, 11):
    nmf = NMF(
        n_components=num_topics,
        init='nndsvd', 
        random_state = 42
        )
    W = nmf.fit_transform(tfidf_documents_NMF)
    H = nmf.components_
    top_components = np.argsort(-H)[:, :10]
    
    topics = []
    for t in top_components:
        topic_words = []
        for w in t:
            topic_words.append(feature_names_NMF[w])
        topics.append(topic_words)
        
    print(f"{num_topics} Topics:")
    print(topics)
    print("\n")
        

2 Topics:
[['family', 'young', 'old', 'father', 'year', 'woman', 'year old', 'home', 'mother', 'finds'], ['life', 'new', 'film', 'story', 'world', 'documentary', 'york', 'new york', 'city', 'love']]


3 Topics:
[['family', 'young', 'old', 'father', 'woman', 'year', 'home', 'year old', 'mother', 'finds'], ['life', 'new', 'film', 'story', 'world', 'documentary', 'york', 'new york', 'city', 'love'], ['school', 'high', 'high school', 'students', 'senior', 'team', 'student', 'school football', 'school students', 'football']]


4 Topics:
[['family', 'young', 'father', 'woman', 'home', 'mother', 'son', 'wife', 'daughter', 'house'], ['life', 'new', 'film', 'story', 'world', 'documentary', 'york', 'new york', 'love', 'city'], ['school', 'high', 'high school', 'students', 'team', 'senior', 'student', 'school football', 'school students', 'football'], ['year', 'old', 'year old', 'old girl', 'finds', 'love', 'old daughter', 'girl', 'best', 'friend']]


5 Topics:
[['family', 'father', 'young', 'wom

------

### NYT Full Review Text

TFIDF and NMF on full NYT Review text

In [13]:
NYTReviews = pd.read_csv("data/NYTData_wReviewText.csv")

In [70]:
tfidf_vectorizer_NMF = TfidfVectorizer(max_features = 20000, # only top 5k by freq
                                       lowercase = True, # drop capitalization
                                       ngram_range = (1,2), # include up to 2-grams, we can make this only 1 if needed
                                       min_df=1,  # note: absolute count of doc
                                       token_pattern = r'\b[a-z]{3,12}\b',   # remove short, non-word-like terms
                                       stop_words='english') # default English stopwords

tfidf_documents_NMF = tfidf_vectorizer_NMF.fit_transform(NYTReviews["review_text"])
feature_names_NMF = tfidf_vectorizer_NMF.get_feature_names_out()

In [71]:
from sklearn.decomposition import NMF
for num_topics in range(2, 11):
    nmf = NMF(
        n_components=num_topics,
        init='nndsvd', 
        random_state = 42
        )
    W = nmf.fit_transform(tfidf_documents_NMF)
    H = nmf.components_
    top_components = np.argsort(-H)[:, :10]
    
    topics = []
    for t in top_components:
        topic_words = []
        for w in t:
            topic_words.append(feature_names_NMF[w])
        topics.append(topic_words)
        
    print(f"{num_topics} Topics:")
    print(topics)
    print("\n")

2 Topics:
[['movie', 'like', 'story', 'film', 'character', 'time', 'characters', 'family', 'man', 'life'], ['documentary', 'film', 'says', 'footage', 'new', 'people', 'interviews', 'work', 'black', 'world']]


3 Topics:
[['movie', 'like', 'story', 'film', 'character', 'time', 'characters', 'family', 'man', 'life'], ['documentary', 'film', 'says', 'footage', 'people', 'new', 'interviews', 'black', 'work', 'history'], ['oscar', 'oscar nominated', 'nominated', 'live action', 'short', 'colette', 'program', 'program hour', 'hour program', 'action']]


4 Topics:
[['movie', 'like', 'story', 'character', 'characters', 'film', 'time', 'family', 'man', 'woman'], ['film', 'documentary', 'says', 'people', 'camera', 'history', 'subjects', 'world', 'women', 'interviews'], ['oscar', 'oscar nominated', 'nominated', 'live action', 'short', 'colette', 'program', 'hour program', 'program hour', 'action'], ['music', 'new', 'rock', 'york', 'new york', 'musical', 'artist', 'band', 'movie', 'documentary']]



------

# Turn Topics into Features

## NMF Helper Functions

In [83]:
# train a tfidf bag of words vectorizer
def tfidf_vecorization(data, ngram_range):
    tfidf_vectorizer_NMF = TfidfVectorizer(max_features = 20000, # only top 20k by freq
                                       lowercase = True, # drop capitalization
                                       ngram_range = ngram_range, # include up to 2-grams, we can make this only 1 if needed
                                       min_df=1,  # note: absolute count of doc
                                       token_pattern = r'\b[a-z]{3,}\b',   # remove short, non-word-like terms
                                       stop_words='english') # default English stopwords

    tfidf_documents_NMF = tfidf_vectorizer_NMF.fit_transform(data)
    feature_names_NMF = tfidf_vectorizer_NMF.get_feature_names_out()
    
    return tfidf_documents_NMF, feature_names_NMF

In [114]:
# take TFIDF transformed data and train NMF
def trainNMF(tfidf, n_components):
    nmf = NMF(
        n_components=n_components,
        init='nndsvd', 
        random_state = 42
        )
    W = nmf.fit_transform(tfidf_documents_NMF)
    H = nmf.components_
    top_components = np.argsort(-H)[:, :10]

    topics = []
    for t in top_components:
        topic_words = []
        for w in t:
            topic_words.append(feature_names_NMF[w])
        topics.append(topic_words)
        
    return W, H, topics

## Load Movie Data

In [124]:
movie_data = pd.read_csv("data/joined_df.csv")
movie_data.Plot = movie_data.Plot.fillna(value="")

## Assign each movie a review topic

Using the W matrix, we can see which topic is most related to that movie's review.

In [127]:
tfidf_documents_NMF, feature_names_NMF = tfidf_vecorization(movie_data['review_text'], (1,2))
W, H, topics = trainNMF(tfidf_documents_NMF, 5)

In [128]:
# W is the topic matrix, the highest value within each row will determine the most relevant topic
movie_data["review_topic"] = [x.argmax() for x in W]

## Assign each movie a plot topic

Using the W matrix, we can see which topic is most related to that movie's plot. (Plot topics are different than review topics)

In [130]:
tfidf_documents_NMF, feature_names_NMF = tfidf_vecorization(movie_data['Plot'], (1,2))
W, H, topics = trainNMF(tfidf_documents_NMF, 5)

In [132]:
# W is the topic matrix, the highest value within each row will determine the most relevant topic
movie_data["plot_topic"] = [x.argmax() for x in W]

## Preview Data

In [134]:
movie_data[["Title", "Rated", "review_topic", "plot_topic"]].head(10)

,Title,Rated,review_topic,plot_topic
0,Tenet,PG-13,0,1
1,Dune,PG-13,0,1
2,Zack Snyder's Justice League,R,0,0
3,Soul,PG,4,2
4,Soul,PG,0,2
5,The Gentlemen,R,0,1
6,Black Widow,PG-13,0,1
7,The Suicide Squad,R,0,4
8,Shang-Chi and the Legend of the Ten Rings,PG-13,0,0
9,Wonder Woman 1984,PG-13,0,0


In [135]:
movie_data.to_csv("data/movie_data_withTopics")